In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

from chains import get_llm


In [2]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = get_llm(api_key="OLLAMA")  
generate = prompt | llm

In [3]:
essay = ""

request = HumanMessage(
    content="Write an essay explaining the central theme of the count of Monte Cristo"
)

for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content


Certainly! Here is a five-paragraph essay exploring the central theme of "The Count of Monte Cristo" by Alexandre Dumas:

### The Central Theme of "The Count of Monte Cristo"

"The Count of Monte Cristo," or "The Count of the Mont-Croix" as it was originally titled, is an epic tale that delves deeply into themes of revenge and redemption. At its core lies a compelling exploration of how individuals can transform their lives through perseverance and forgiveness in response to unjust treatment.

At first glance, Edmond Dantès's story appears to be one of vengeance against those who betrayed him. His imprisonment by the machinations of his treacherous political allies is the catalyst for his journey into the depths of the labyrinthine Monte Cristo Castle. However, as he navigates this castle and encounters various characters, including a mysterious man named Fernand, Dantès's perspective begins to shift. He realizes that his initial desire for vengeance has blinded him to the suffering he